# RAVDESS Song Emotion Recognition

In [1]:
import glob
import os
import librosa
import numpy as np
import tensorflow as tf

In [2]:
!git clone https://github.com/bagustris/ravdess_song.git

Cloning into 'ravdess_song'...
remote: Enumerating objects: 1165, done.
remote: Counting objects: 100% (108/108), done.
remote: Compressing objects: 100% (96/96), done.
remote: Total 1165 (delta 61), reused 32 (delta 12), pack-reused 1057
Receiving objects: 100% (1165/1165), 232.31 MiB | 23.04 MiB/s, done.
Resolving deltas: 100% (65/65), done.
Checking out files: 100% (1030/1030), done.


In [3]:
cd ravdess_song

/content/ravdess_song


In [4]:
# load extracted features
x_train = np.load('data/x_train.npy', allow_pickle=True)
x_test = np.load('data/x_test.npy', allow_pickle=True)
y_train = np.load('data/y_train.npy', allow_pickle=True)
y_test = np.load('data/y_test.npy', allow_pickle=True)


In [5]:
# reshape X for CNN
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)


In [6]:
def model_cnn():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.BatchNormalization(axis=-1,
              input_shape=(x_train.shape[1], x_train.shape[2])))
    model.add(tf.keras.layers.Conv1D(256, 128, 1, padding='same'))
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.MaxPooling1D(2))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dropout(0.4))
    model.add(tf.keras.layers.Dense(6, activation='softmax'))

    # compile model: set loss, optimizer, metric
    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                  optimizer=tf.keras.optimizers.Adam(),
                  metrics=['accuracy'])
    return model

In [7]:
model = model_cnn()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 386, 1)            4         
_________________________________________________________________
conv1d (Conv1D)              (None, 386, 256)          33024     
_________________________________________________________________
activation (Activation)      (None, 386, 256)          0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 193, 256)          0         
_________________________________________________________________
flatten (Flatten)            (None, 49408)             0         
_________________________________________________________________
dropout (Dropout)            (None, 49408)             0         
_________________________________________________________________
dense (Dense)                (None, 6)                 2

In [8]:
# train CNN model
hist = model.fit(x_train, y_train, epochs=100, shuffle=True, batch_size=16)

Epoch 1/100
53/53 [==============================] - 4s 63ms/step - loss: 1.7686 - accuracy: 0.2512
Epoch 2/100
53/53 [==============================] - 3s 63ms/step - loss: 1.5629 - accuracy: 0.3648
Epoch 3/100
53/53 [==============================] - 3s 64ms/step - loss: 1.4139 - accuracy: 0.4354
Epoch 4/100
53/53 [==============================] - 3s 63ms/step - loss: 1.2030 - accuracy: 0.5239
Epoch 5/100
53/53 [==============================] - 3s 63ms/step - loss: 1.1417 - accuracy: 0.5526
Epoch 6/100
53/53 [==============================] - 3s 64ms/step - loss: 1.0603 - accuracy: 0.5873
Epoch 7/100
53/53 [==============================] - 3s 63ms/step - loss: 1.0191 - accuracy: 0.6256
Epoch 8/100
53/53 [==============================] - 3s 64ms/step - loss: 0.8336 - accuracy: 0.6842
Epoch 9/100
53/53 [==============================] - 3s 64ms/step - loss: 0.7931 - accuracy: 0.7069
Epoch 10/100
53/53 [==============================] - 3s 64ms/step - loss: 0.8004 - accuracy: 0.7045

In [9]:
evaluate = model.evaluate(x_test, y_test, batch_size=16)
print("Loss: ", evaluate[0], "--> Accuracy: ", evaluate[1])

11/11 [==============================] - 0s 15ms/step - loss: 0.8452 - accuracy: 0.7500
Loss:  0.8451672196388245 --> Accuracy:  0.75
